In [1]:
import os

In [2]:
%pwd

'c:\\Projects\\End-to-End-chest-classification-using-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Projects\\End-to-End-chest-classification-using-DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml , create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [8]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: 'DataIngestionConfig'):
        self.config = config

    def download_file(self) -> str:
        """
        Fetch data from the URL and save it locally as a zip file.
        """
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file

            # Create the parent directory for the zip file if it doesn't exist
            os.makedirs(os.path.dirname(zip_download_dir), exist_ok=True)

            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            # Correcting the prefix for Google Drive file download
        
            file_url = 'https://drive.google.com/uc?id=11JqNDxjF-pQC_jGXDwWG0djAGJGHkdNs&export=download'
            
            # Downloading the file using gdown
            gdown.download(file_url, zip_download_dir, quiet=False)  # Set quiet=False for detailed logs

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            logger.error(f"Error occurred while downloading file: {e}")
            raise e

    def extract_zip_file(self):
        """
        Extracts the zip file into the specified directory.
        """
        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)

            # Extracting the zip file
            with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
                zip_ref.extractall(unzip_path)
            
            logger.info(f"Extracted zip file {self.config.local_data_file} into directory {unzip_path}")
        except zipfile.BadZipFile as e:
            logger.error(f"Invalid zip file: {e}")
            raise e
        except FileNotFoundError as e:
            logger.error(f"Zip file not found: {e}")
            raise e
        except Exception as e:
            logger.error(f"Error occurred while extracting zip file: {e}")
            raise e

In [10]:
try:
    logger.info("Initializing ConfigurationManager...")
    config = ConfigurationManager()
    
    logger.info("Fetching DataIngestionConfig...")
    data_ingestion_config = config.get_data_ingestion_config()
    
    logger.info("Initializing DataIngestion...")
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    logger.info("Downloading file...")
    data_ingestion.download_file()
    
    logger.info("Extracting zip file...")
    data_ingestion.extract_zip_file()
    
    logger.info("Pipeline executed successfully!")
except Exception as e:
    logger.error(f"Pipeline execution failed: {e}")
    raise RuntimeError(f"Pipeline execution failed due to: {str(e)}") from e


[2024-12-22 00:40:31,164: INFO: 3649388521: Initializing ConfigurationManager...]
[2024-12-22 00:40:31,176: INFO: common: YAML file: config\config.yaml loaded successfully]
[2024-12-22 00:40:31,176: INFO: common: YAML file: params.yaml loaded successfully]
[2024-12-22 00:40:31,185: INFO: common: Created directory at: artifacts]
[2024-12-22 00:40:31,185: INFO: 3649388521: Fetching DataIngestionConfig...]
[2024-12-22 00:40:31,187: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-12-22 00:40:31,187: INFO: 3649388521: Initializing DataIngestion...]
[2024-12-22 00:40:31,187: INFO: 3649388521: Downloading file...]
[2024-12-22 00:40:31,191: INFO: 2636377133: Downloading data from https://drive.google.com/uc?id=11JqNDxjF-pQC_jGXDwWG0djAGJGHkdNs&export=download into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?id=11JqNDxjF-pQC_jGXDwWG0djAGJGHkdNs&export=download
From (redirected): https://drive.google.com/uc?id=11JqNDxjF-pQC_jGXDwWG0djAGJGHkdNs&export=download&confirm=t&uuid=74ac07c6-07e4-4562-bd76-9ea715f23e91
To: c:\Projects\End-to-End-chest-classification-using-DVC\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:05<00:00, 8.20MB/s]

[2024-12-22 00:40:41,623: INFO: 2636377133: Downloaded data from https://drive.google.com/uc?id=11JqNDxjF-pQC_jGXDwWG0djAGJGHkdNs&export=download into file artifacts/data_ingestion/data.zip]
[2024-12-22 00:40:41,623: INFO: 3649388521: Extracting zip file...]


[2024-12-22 00:40:42,321: INFO: 2636377133: Extracted zip file artifacts/data_ingestion/data.zip into directory artifacts/data_ingestion]
[2024-12-22 00:40:42,321: INFO: 3649388521: Pipeline executed successfully!]
